In [1]:
from binance.spot import Spot
from binance.websocket.spot.websocket_api import SpotWebsocketAPIClient
from binance.websocket.spot.websocket_stream import SpotWebsocketStreamClient
from tqdm.autonotebook import tqdm
from utils.utils import build_dataset, build_market_image, preprocess_data,symbol_map,sharpe_reward
import pandas as pd
from gym_trading_env.downloader import download
from gym_trading_env.utils.history import History
from gym_trading_env.utils.portfolio import Portfolio, TargetPortfolio
import datetime
import glob
import numpy as np
import sqlite3 as db
import time
import datetime
from neuralforecast.core import NeuralForecast
from MultiTrade.environments import NeuralForecastingTradingEnv,LiveTradingEnv
import warnings
warnings.filterwarnings('ignore')
from ray.rllib.algorithms.algorithm import Algorithm
from ray.tune.registry import register_env
import sqlite3 as db

from IPython. display import display

C:\Users\standard\AppData\Local\Temp\ipykernel_60748\3683853900.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
image=build_market_image(target_pair='ETH/USDT',time_frame='1h',axis=0)

In [3]:
datetime. datetime.now()-pd.Timedelta('1h').to_pytimedelta()

datetime.datetime(2024, 4, 26, 6, 14, 20, 933581)

In [4]:
trade_target='BTC/USDT'
api_key='ZGvDtoVQM9O1xb0PXaSPzMffWtGH3ofxVSbsuzGQNDBwklCHkKm4x1g3nrImqQXF'
api_secret='BUDui1xmoP11wGmSV3RqTrtrkmYU5EyogHMtigVQgTGoQxjcibkuwjN5Et169f4o'

client =Spot(api_key=api_key,api_secret=api_secret,base_url='https://testnet.binance.vision')
client


In [5]:
# account=client.account()

In [6]:
# client.avg_price('ETHUSDT'),client.ticker_price('ETHUSDT')


In [7]:

model=NeuralForecast.load("C:/Users/standard/Git/MultiTrader/forecaster_pretrain/neuralForecast")

Seed set to 1
Seed set to 1
Seed set to 1


In [8]:
live_env_config=dict(
                name='ETHUSDT_train',
                model=model,
                api_key=api_key,
                api_secret=api_secret,
                test_net=True,
                restore_trading=False,
                target_symbol='ETH/USDT',
                time_frame='1h',
                reward_function=sharpe_reward,
                positions = [ -.5,-.25,.25, .5], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                max_episode_duration=168,
                verbose=0,
                )

In [9]:
self=LiveTradingEnv(**live_env_config)
self.df

,open,high,low,close,volume,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,...,feature_log_volume_20,feature_log_return_25,feature_log_volume_25,feature_log_return_30,feature_log_volume_30,ds,symbol,unique_id,dynamic_feature__0,dynamic_feature__1
date_close,,,,,,,,,,,,,,,,,,,,,
2024-04-23 14:59:59.999,3176.969664,3261.212108,3192.878325,3232.715073,217.073978,3232.715073,0.608696,0.333333,-2.1,22.976256,...,0.527503,0.808189,0.564927,0.887166,0.750357,2024-04-23 14:59:59.999,ETHUSDT,8,0,0
2024-04-23 15:59:59.999,3232.319664,4863.752108,3150.458325,3210.595073,237.855078,3210.595073,0.652174,0.333333,-2.1,22.976256,...,0.527503,0.808189,0.564927,0.887166,0.750357,2024-04-23 15:59:59.999,ETHUSDT,8,0,0
2024-04-23 16:59:59.999,3210.439664,3242.552108,3217.368325,3216.245073,169.812478,3216.245073,0.695652,0.333333,-2.1,22.976256,...,0.527503,0.808189,0.564927,0.887166,0.750357,2024-04-23 16:59:59.999,ETHUSDT,8,0,0
2024-04-23 17:59:59.999,3215.189664,3243.402108,3225.218325,3226.945073,166.075978,3226.945073,0.739130,0.333333,-2.1,22.976256,...,0.527503,0.808189,0.564927,0.887166,0.750357,2024-04-23 17:59:59.999,ETHUSDT,8,0,0
2024-04-23 18:59:59.999,3226.859664,3247.652108,3220.078325,3208.445073,174.348078,3208.445073,0.782609,0.333333,-2.1,22.976256,...,0.527503,0.808189,0.564927,0.887166,0.750357,2024-04-23 18:59:59.999,ETHUSDT,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-26 00:59:59.999,3140.959664,3309.062108,3133.968325,3126.825073,108.683778,3126.825073,0.000000,0.833333,-2.0,22.984475,...,0.309904,0.809443,-0.014214,0.891598,0.249762,2024-04-26 00:59:59.999,ETHUSDT,8,0,0
2024-04-26 01:59:59.999,3126.859664,3150.132108,3125.768325,3135.565073,134.004178,3135.565073,0.043478,0.833333,-2.0,22.984475,...,0.114413,0.805312,0.265906,0.889580,-0.658777,2024-04-26 01:59:59.999,ETHUSDT,8,0,0
2024-04-26 02:59:59.999,3135.459664,3342.162108,3088.748325,3139.025073,129.019178,3139.025073,0.086957,0.833333,-2.0,22.984475,...,0.351348,0.807732,0.183101,0.893949,0.179200,2024-04-26 02:59:59.999,ETHUSDT,8,0,0


In [10]:
obs,info=self.reset(reset_account=False)
obs.shape


(51,)

In [11]:
from ray.rllib.policy.policy import Policy

In [12]:
agent= Policy.from_checkpoint('C:/Users/standard/Git/MultiTrader/Rayrl_tune/checkpoints/policies/default_policy')

agent.export_model('C:/Users/standard/Git/MultiTrader/Rayrl_tune/checkpoints/pretrained_agent')


In [13]:
# action,_,states=agent.compute_single_action(obs)
# obs, reward, terminated, truncated, info=self.live_step(action,wait=False)
# action,_,states=agent.compute_single_action(obs)


In [14]:
obs,info=self.reset(reset_account=True)
while True:
    action,_,states=agent.compute_single_action(obs)
    obs, reward, terminated, truncated, info=self.live_step(action,wait=True)

reset_account
{'symbol': 'ETHUSDT', 'side': 'BUY', 'type': 'MARKET', 'quoteOrderQty': 5228.2486005}
 0.5
portfoliio {'asset': 1.9094, 'fiat': 5975.405565, 'borrowed_asset': 0, 'borrowed_fiat': 0, 'interest_asset': 0, 'interest_fiat': 0, 'valuation': 11952.018505}
Trade 0->0.5
              0.5, so 2987.7USDT/ 1.9090142695121561ETH,
              BUY ETH:1.9090142695121561 for 2987.70$ at 3130.1 ETHUSDT
{'symbol': 'ETHUSDT', 'side': 'BUY', 'type': 'MARKET', 'quoteOrderQty': 2987.7027825}


  0%|          | 0/2723 [00:00<?, ?it/s]

In [ ]:
self.df.iloc[-1]

In [ ]:
# h=self.historical_info[0]
history_df=pd.DataFrame([self.historical_info[-1]])
history_df

In [ ]:
import sqlite3 as db




# connect to a test DB with one three-column table titled "Demo"
conn = db.connect('Trade_history/trade.db')
cur = conn .cursor()

In [ ]:
history_df.to_sql(f'{self.symbol}_trade_history',conn,if_exists='append')

In [ ]:
history_df=pd.read_sql(f'select * from {self.symbol}_trade_history',conn)
history_df

In [ ]:
self.historical_info[-1]

In [ ]:
for h in self.historical_info:
    insert_sql = f'INSERT INTO {self.symbol}_trade_history {h}' 
    conn.execute(insert_sql)

In [ ]:
client.klines("BNBUSDT", "1h", limit=1)

In [ ]:
self._get_ticker()

In [ ]:
data_list=client.klines("BNBUSDT", "1h", limit=10)
columns=['date_open','open','high','low','close','volume','date_close','QA_volume','N_trades','BA_volume','BQ_volume','unused']
data=pd.DataFrame(data_list,columns=columns)
data['date_close']=data['date_close'].apply(pd.to_datetime,unit='ms')
data['date_open']=data['date_open'].apply(pd.to_datetime,unit='ms')

# data_list

In [ ]:
data